# 基于LangChain的文档问答
本章内容主要利用langchain构建向量数据库，可以在文档上方或关于文档回答问题，因此，给定从PDF文件、网页或某些公司的内部文档收集中提取的文本，使用llm回答有关这些文档内容的问题

## 环境配置
安装langchain，设置chatGPT的OPENAI_API_KEY

 - 安装langchain

    ```
    pip install langchain
    ```  
 
 - 安装docarray

    ```
    pip install docarray
    ```  
 
 - 设置API-KEY环境变量

    ```  
    export BAICHUAN_API_KEY='api-key'
    ```


In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) #读取环境变量

In [2]:
from langchain_community.chat_models import ChatBaichuan
llm = ChatBaichuan(temperature=0.9)
llm.invoke("1+1=？")   # 证明模型可以使用

AIMessage(content='1+1=2', response_metadata={'token_usage': {'prompt_tokens': 7, 'completion_tokens': 6, 'total_tokens': 13}, 'model': 'Baichuan2-Turbo-192K'}, id='run-652f153b-2a85-4a4e-bfc1-15ca0afb213d-0')

## 导入embedding模型和向量存储组件

使用Dock Array内存搜索向量存储，作为一个内存向量存储，不需要连接外部数据库

In [3]:
from langchain.chains import RetrievalQA #检索QA链，在文档上进行检索
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.vectorstores import DocArrayInMemorySearch #向量存储
from IPython.display import display, Markdown #在jupyter显示信息的工具

In [5]:
# import pandas as pd
# names = ["Akuce","Bob","Charlie","David","Eva","Frank"]
# descs = [
#     "是一位优秀的科学家，专注于研究人工智能。",
#     "是一位出色的艺术家，擅长绘画和雕塑。",
#     "是一位杰出的企业家，创办了多家成功的公司。",
#     "是一位卓越的音乐家，精通多种乐器。",
#     "是一位卓越的运动员，多次获得世界冠军。",
#     "是一位杰出的作家，创作了多部畅销书籍。",
# ]
# file = "people.csv"
# df = pd.DataFrame({"name":names,"description":descs})
# df.to_csv(file,index=False)
# loader = CSVLoader(file_path=file,encoding='utf8')

## index和query

In [6]:
file = "people.csv"
loader = CSVLoader(file_path=file,encoding='utf8')
#查看数据
import pandas as pd
data = pd.read_csv(file)
print(data)

      name            description
0    Akuce   是一位优秀的科学家，专注于研究人工智能。
1      Bob     是一位出色的艺术家，擅长绘画和雕塑。
2  Charlie  是一位杰出的企业家，创办了多家成功的公司。
3    David      是一位卓越的音乐家，精通多种乐器。
4      Eva    是一位卓越的运动员，多次获得世界冠军。
5    Frank    是一位杰出的作家，创作了多部畅销书籍。


提供了一个简单的CSV文件，我们将使用它与语言模型结合使用

## 创建向量存储 

将导入一个索引，即向量存储索引创建器

In [7]:
from langchain_community.embeddings import BaichuanTextEmbeddings
import os

embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])

In [8]:
from langchain.indexes import VectorstoreIndexCreator #导入向量存储索引创建器

从langchain.indexes模块中导入VectorstoreIndexCreator类，这个类用于创建基于向量存储的索引。

接下来，实例化一个VectorstoreIndexCreator对象，命名为index_creator。在这个过程中，传入了两个参数：

 - embedding参数：这里传入了一个名为embeddings的变量，它应该是之前定义的一个embedding模型或者对象。这个模型负责将文本数据转化为高维向量，以便于之后在向量空间中进行相似性搜索。
 - vectorstore_cls参数：设置为DocArrayInMemorySearch类，这指定了所使用的向量数据库类型。DocArrayInMemorySearch是一种在内存中存储和搜索向量数据的实现方式，适用于数据量不大且追求快速检索的场景。这意味着创建的索引将会存储在内存中，而非持久化到磁盘，这样可以提高查询速度，但可能会占用较多的内存资源。

In [10]:
index_creator = VectorstoreIndexCreator(
    embedding = embeddings,
    vectorstore_cls=DocArrayInMemorySearch
)

In [11]:
'''
将指定向量存储类,创建完成后，我们将从加载器中调用,通过文档记载器列表加载
'''
index = index_creator.from_loaders([loader])

C:\Users\Administrator\miniconda3\envs\langchain-web-1\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [12]:
query ="谁喜欢计算机，为什么呢？"

我们可以使用自定义提示模板修改查询以满足我们的需求

In [13]:
from langchain.prompts import PromptTemplate
prompt_template = """使用以下上下文来回答最后的问题。
如果您不知道答案，请理性思考并根据自己的知识库进行回答

{context}

Question: {question}
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [15]:
response = index.query(query,llm=llm, chain_type_kwargs=chain_type_kwargs)  #使用索引查询创建一个响应，并传入这个查询
response

'根据上下文，Akuce 是一位专注于研究人工智能的科学家，因此他可能会喜欢计算机。科学家通常对新技术和设备感兴趣，特别是那些有助于他们进行研究和创新的工具。Akuce 可能会喜欢计算机，因为它们是他在人工智能领域工作的重要工具。'

In [17]:
docs = loader.load()
docs[0]

Document(page_content='name: Akuce\ndescription: 是一位优秀的科学家，专注于研究人工智能。', metadata={'source': 'people.csv', 'row': 0})

## Embedding

In [18]:
embed = embeddings.embed_query("我喜欢研究算法")

In [19]:
embed[:5]

[0.04571808, 0.02734232, -0.0057214918, 0.028426658, -0.028289212]

## 相似文档查找

In [22]:
db = DocArrayInMemorySearch.from_documents(
    docs,
    embeddings
)
docs = db.similarity_search("我喜欢弹琴")

In [23]:
docs

[Document(page_content='name: David\ndescription: 是一位卓越的音乐家，精通多种乐器。', metadata={'source': 'people.csv', 'row': 3}),
 Document(page_content='name: Bob\ndescription: 是一位出色的艺术家，擅长绘画和雕塑。', metadata={'source': 'people.csv', 'row': 1}),
 Document(page_content='name: Eva\ndescription: 是一位卓越的运动员，多次获得世界冠军。', metadata={'source': 'people.csv', 'row': 4}),
 Document(page_content='name: Akuce\ndescription: 是一位优秀的科学家，专注于研究人工智能。', metadata={'source': 'people.csv', 'row': 0})]

In [24]:
retriever= db.as_retriever()
llm = ChatBaichuan(temperature=0.0)
qdocs = "".join([docs[i].page_content for i in range(len(docs))])
qdocs

'name: David\ndescription: 是一位卓越的音乐家，精通多种乐器。name: Bob\ndescription: 是一位出色的艺术家，擅长绘画和雕塑。name: Eva\ndescription: 是一位卓越的运动员，多次获得世界冠军。name: Akuce\ndescription: 是一位优秀的科学家，专注于研究人工智能。'

In [26]:
resp = llm.invoke(f"{qdocs} 问题：请列出跟科学艺术相关的人")
print(resp.content)

根据您提供的信息，与科学和艺术相关的人包括：

1. Bob：一位出色的艺术家，擅长绘画和雕塑。
2. Akuce：一位优秀的科学家，专注于研究人工智能。


In [27]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, please think rationally and answer from your own knowledge base 

{context}

Question: {question}
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True,
    chain_type_kwargs=chain_type_kwargs
)
query = "喜欢弹琴的是谁，为什么？"
resp = qa_stuff.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [28]:
resp

{'query': '喜欢弹琴的是谁，为什么？',
 'result': '根据描述，David是一位卓越的音乐家，精通多种乐器。因此，喜欢弹琴的人应该是David。'}

In [29]:
query = "请列出跟科学艺术相关的人？"
resp = qa_stuff.invoke(query)
resp



> Entering new RetrievalQA chain...

> Finished chain.


{'query': '请列出跟科学艺术相关的人？',
 'result': '跟科学艺术相关的人有：\n1. Bob：是一位出色的艺术家，擅长绘画和雕塑。\n2. Akuce：是一位优秀的科学家，专注于研究人工智能。'}